In [23]:
import networkx as nx
import freeman as fm
import matplotlib.pyplot as plt
from pprint import pprint
import pymysql
import sys

In [24]:
from setup import mysql_setup
conn = mysql_setup()

In [25]:
g = nx.Graph()


In [26]:
with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM Playlist")
        playlists = cursor.fetchall()
    except pymysql.err.IntegrityError as e:
        print("Erro: não foi possivel dar select em playlist")
        pass

In [27]:
pprint(playlists)

(('01pNIDYGqmeawppy32wr3D', '2000s Hip Hop', 'warnermusicus', 0, 1),
 ('37i9dQZF1DWT5MrZnPU1zD', 'Hip Hop Controller', 'spotify', 0, 1),
 ('37i9dQZF1DWTVIaV4KJYuq', 'Classic Covers', 'spotify', 0, 1),
 ('37i9dQZF1DWTwnEm1IYyoj', 'Soft Pop Hits', 'spotify', 0, 1),
 ('37i9dQZF1DWU6MDxlH7Sr1', 'Cocktail Jazz', 'spotify', 0, 1),
 ('37i9dQZF1DWUa8ZRTfalHk', 'Pop Rising', 'spotify', 0, 1),
 ('37i9dQZF1DWUFmyho2wkQU', 'Hip-Hop Drive', 'spotify', 0, 1),
 ('37i9dQZF1DWVLcZxJO5zyf', 'Pop Brasil', 'spotify', 0, 1),
 ('37i9dQZF1DWVqfgj8NZEp1', 'Coffee Table Jazz', 'spotify', 0, 1),
 ('37i9dQZF1DWXJnyndhASBe', 'Generacja Hip Hop', 'spotify', 0, 1),
 ('37i9dQZF1DWXRqgorJj26U', 'Rock Classics', 'spotify', 0, 1),
 ('37i9dQZF1DWXTHBOfJ8aI7', 'Légendes du Rock', 'spotify', 0, 1),
 ('37i9dQZF1DWY6tYEFs22tT', 'Hip-Hop Central', 'spotify', 0, 1),
 ('37i9dQZF1DWYDQ8wBxd7xt', 'Pinoy Classic Rock', 'spotify', 0, 1),
 ('37i9dQZF1DWYNSm3Z3MxiM', 'Classic Rock Workout', 'spotify', 0, 1),
 ('37i9dQZF1DWZeuoGxfO4e

In [28]:
def return_tracks(id_playlist):
    with conn.cursor() as cursor:
        try:
            cursor.execute("SELECT * FROM Playlist_Track WHERE id_playlist = %s", (id_playlist))
            tracks = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel dar select em playlist")
            sys.exit()
    return tracks

In [29]:
for playlist in playlists:
    g.add_node(playlist[1])

for playlists_index1 in range(len(playlists)-1):
    tracks_1 = return_tracks(playlists[playlists_index1][0])
    
    if len(tracks_1) == 0:
        continue
        
    for playlists_index2 in range(len(playlists)-1-playlists_index1):
        tracks_2 = return_tracks(playlists[playlists_index1+playlists_index2+1][0])
        
        if len(tracks_2) == 0:
            continue

        count = 0
        
        for t1 in tracks_1:
            for t2 in tracks_2:
                if t1[1] == t2[1]:
                    count += 1
    
        g.add_edge(playlists[playlists_index1][1], playlists[playlists_index1+playlists_index2+1][1] , weight=count)

In [33]:
nx.write_gml(g, "test.gml")

In [34]:
g = fm.load('test.gml')

In [38]:
g.set_all_nodes('labpos', 'hover')
g.move('kamada_kawai')
g.interact()